## Objective: To subset the stage IV radar precipitation data for NOAA Atlas 14 Volume 13 region

#### The grid is based on a polar stereo graphic map projection with a standard latitude of 60° North and standard longitude of 105° West.
#### The mesh length at 60° North latitude is 4.7625 KM. (Source: https://www.weather.gov/media/owp/oh/hrl/docs/21hrapgrid.pdf)

1. It is a 1121x881 polar stereographic grid.
2. Point (1,1) is at 23.117N 119.023W.
3. The y-axis is parallel to 105W.
4. The resolution is 4.7625km at 60N.
5. The pole point is (I,J) = (400.5,1600.5) 

In [1]:
import os
import numpy as np
import xarray as xr
import nctoolkit as nc
import geopandas as gpd

nctoolkit is using Climate Data Operators version 2.1.1


In [2]:
nc.options(lazy = False) # This will compute each operation.

In [3]:
# FUnction to get the file size
def get_file_size(file_path):
    size = os.path.getsize(file_path)  # Size in bytes
    size_in_mb = size / (1024 * 1024)
    return size_in_mb

In [4]:
# Load shapefile into a geopandas dataframe
shapefile = gpd.read_file('shapefile/na14vol13_buffer_1p20deg.shp')
lon_min, lat_min, lon_max, lat_max = shapefile.total_bounds # Get latitude and longitude bounds [i.e four corners of the box]

# Print the file size
filepath = 'data/2013_stage4_daily.nc'
print(f'The file size for stage IV data: {get_file_size(filepath)} MB')

# Load stage 4 data
data = nc.open_data(filepath)

# subset the data based on latitude bounds of vol13 shapefile
data.subset(lon = [lon_min, lon_max], lat = [lat_min, lat_max])

# Regrid a dataset to a regular latlon grid
resolution = 0.05 # resolution to regrid in degrees
data.to_latlon(lon = [lon_min, lon_max], lat = [lat_min, lat_max], res=resolution)

# convert the data to xarray dataset
dataxr = data.to_xarray()

precip = dataxr['p01d_12z']

The file size for stage IV data: 2765.286639213562 MB


In [5]:
# Create a mask of the same shape as the data
nan_mask = np.isnan(precip)

# Assign the data intervals to digitize.
bins = np.array([0.01, 0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0])
bin_indices = np.digitize(precip, bins, right=True) # setting right to True means the NaNs are replaced by 0

# Convert bin_indices to int8 type. 
bin_indices = bin_indices.astype(np.int8)

# Use mask to replace the bin indices of NaN values in original data with -1 or any flag value
bin_indices[nan_mask] = -1  # -1 is missing value.

# delete the original variable
del dataxr['p01d_12z']

# Create a new data variable in the xarray dataset with the binned data
dataxr['p01d_12z'] = xr.DataArray(bin_indices, dims=precip.dims, coords=precip.coords)

# Assign chunk to the dataset. Chuning is done only along the time dimension. 
dataxr = dataxr.chunk({'time': 60, 'lat': -1, 'lon': -1})

# Add a missing value attribute
dataxr['p01d_12z'].attrs['missing_value'] = -1

# save the regridded data
outfilepath = 'data/st4_vol13_24h_2013.nc'
dataxr.to_netcdf(outfilepath)

print(f'The file size of the digitized re-grid data: {get_file_size(outfilepath)} MB')

The file size of the digitized re-grid data: 21.275015830993652 MB


In [6]:
dataxr

<xarray.Dataset>
Dimensions:   (time: 365, lon: 241, lat: 253)
Coordinates:
  * time      (time) datetime64[ns] 2013-01-01T12:00:00 ... 2013-12-31T12:00:00
  * lon       (lon) float64 -85.51 -85.46 -85.41 -85.36 ... -73.61 -73.56 -73.51
  * lat       (lat) float64 30.84 30.89 30.94 30.99 ... 43.29 43.34 43.39 43.44
Data variables:
    p01d_12z  (time, lat, lon) int8 dask.array<chunksize=(60, 253, 241), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 2.1.1 (https://mpimet.mpg.de...
    Conventions:  CF-1.0
    source:       Iowa Environmental Mesonet
    institution:  Iowa State University, Ames, IA, USA
    title:        StageIV 12z-12z Totals for 2013
    platform:     Grided
    description:  StageIV
    project_id:   IEM
    contact:      Daryl Herzmann, akrherz@iastate.edu, 515-294-5978
    history:      Tue Aug 01 15:46:46 2023: cdo -L -remap,/tmp/nctoolkit_srid...
    comment:      No Comment at this time
    CDO:          Climate Data Operators version 2.1.1 (https://mpimet.mpg.de...